<a href="https://colab.research.google.com/github/benedettoscala/ifttt-code-generator/blob/main/nl2gpt2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Environment Setup and Library Imports
This section installs the necessary libraries, imports key dependencies, and sets up the workspace.

#### **Dependency Installation**
- The following libraries are installed:
  - `transformers`: Provides pre-trained language models and utilities for NLP tasks.
  - `evaluate`: Enables performance evaluation of language models.
  - `datasets`: Facilitates dataset handling for training and testing.
  - `rouge_score`: (Optional) Used for text-based evaluation metrics, though not strictly needed for GPT-2.

#### **Library Imports**
- **General Libraries:**
  - `os`, `pandas`, `numpy`, and `math` for file handling, data manipulation, and numerical operations.
  - `torch` for deep learning computations.
- **Machine Learning and NLP Libraries:**
  - `sklearn.model_selection` for dataset splitting (train-test split).
  - `datasets.Dataset, DatasetDict` for dataset management.
  - `transformers`:
    - `GPT2LMHeadModel`: Pre-trained GPT-2 model for causal language modeling.
    - `GPT2Tokenizer`: Tokenizer for text processing with GPT-2.
    - `DataCollatorForLanguageModeling`: Ensures proper batching and padding of sequences.
    - `Trainer`: Simplifies training and evaluation of transformers models.
    - `TrainingArguments`: Defines hyperparameters for training.
- **Evaluation Libraries:**
  - `evaluate` for model performance measurement.
  - `nltk` for natural language processing utilities, including tokenization.

#### **NLTK Resource Download**
- If the `punkt` tokenizer is missing, it is downloaded to ensure correct text tokenization.

#### **Repository Cloning**
- The script clones the `ifttt-code-generator` repository from GitHub.
- It navigates to the repository directory and updates it with `git pull`.

This setup ensures a well-prepared environment for training and evaluating language models.


In [ ]:
!pip install transformers evaluate datasets
!pip install rouge_score  # se vuoi ancora utilizzare eventuali metriche di testo, non strettamente necessario per GPT-2


In [ ]:
import os
import pandas as pd
import numpy as np
import torch

from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict
from transformers import (
    GPT2LMHeadModel,
    GPT2Tokenizer,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments
)

import evaluate
import nltk
import math

# Se non hai la risorsa 'punkt' di NLTK installata, esegui:
nltk.download("punkt")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
!git clone https://github.com/benedettoscala/ifttt-code-generator
%cd ifttt-code-generator/
!git pull

Cloning into 'ifttt-code-generator'...
remote: Enumerating objects: 110, done.
remote: Counting objects: 100% (110/110), done.
remote: Compressing objects: 100% (97/97), done.
remote: Total 110 (delta 58), reused 25 (delta 8), pack-reused 0 (from 0)
Receiving objects: 100% (110/110), 14.61 MiB | 18.44 MiB/s, done.
Resolving deltas: 100% (58/58), done.
/content/ifttt-code-generator
Already up to date.


### Dataset Loading and Preprocessing
This section loads, cleans, and prepares the dataset for training a language model.

#### **Dataset Loading and Cleaning**
- The dataset is loaded from a CSV file (`cleaned_and_combined.csv`).
- Rows containing missing values in `cleaned_description` or `filter_code` columns are removed.
- Duplicate entries in these columns are also dropped to ensure unique data points.

#### **Text Formatting for Model Input**
- A **prompt template** is created to format each example:
  - The **description** and **code** are combined into a single string using a separator (`###`).
  - This structure helps the model distinguish between input text (description) and the expected output (code).
  - Example format:
    ```
    Description:
    <description_text>
    ###
    Code:
    <code_text>
    ```
- The `create_text_prompt` function applies this formatting to all dataset entries.

#### **Train/Test Split**
- The dataset is split into:
  - **80% Training Data**
  - **20% Testing Data**
- The split is **random but reproducible** (`random_state=42` ensures consistency across runs).

#### **Conversion to Hugging Face Dataset Format**
- The train and test dataframes are converted into `Dataset` objects for seamless compatibility with the Hugging Face training pipeline.
- A `DatasetDict` structure is created to organize training and testing subsets.

#### **Dataset Size Display**
- The script prints the number of examples in both the training and test sets.

This structured dataset preparation ensures the model receives properly formatted inputs and enables efficient training.


In [ ]:
# Carica il dataset
csv_path = "datasets/cleaned_and_combined.csv"
df = pd.read_csv(csv_path)

# Rimuovi righe con valori mancanti e duplicati
df.dropna(subset=["cleaned_description", "filter_code"], inplace=True)
df.drop_duplicates(subset=["cleaned_description", "filter_code"], inplace=True)

# Esempio: unisci description e code in un'unica stringa.
# Usa un token o un separatore specifico per aiutare il modello a distinguere
# le due parti (es: "###" o "<|endoftext|>")
def create_text_prompt(desc, code):
    return f"Description:\n{desc}\n###\nCode:\n{code}"

df["text"] = df.apply(
    lambda row: create_text_prompt(row["cleaned_description"], row["filter_code"]),
    axis=1
)

# Divisione train/test
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Conversione in Dataset Hugging Face
train_dataset = Dataset.from_pandas(train_df.reset_index(drop=True))
test_dataset = Dataset.from_pandas(test_df.reset_index(drop=True))

dataset = DatasetDict({
    "train": train_dataset,
    "test": test_dataset
})

print("Train set size:", len(dataset["train"]))
print("Test set size:", len(dataset["test"]))

Train set size: 134
Test set size: 34


### Tokenizer Setup and Dataset Tokenization
This section prepares the tokenizer and tokenizes the dataset for training with GPT-2.

#### **Tokenizer Configuration**
- The tokenizer is loaded from the pre-trained `GPT-2` model
  - GPT-2 does not have a default padding token, so the EOS (`end-of-sequence`) token is assigned as the padding token.

#### **Tokenization Function**
- **Input Processing:**
  - Each text sample is tokenized using the GPT-2 tokenizer.
  - `padding="max_length"` ensures all sequences have a fixed size (`256` tokens) for efficient batch processing.
  - `truncation=True` prevents sequences from exceeding the maximum length.
  - **Note:** the longest entry in the dataset is 200 token long.

#### **Tokenizing the Dataset**
- The dataset is processed using `.map()` to apply tokenization to all examples in batches.
- The `remove_columns` parameter ensures only the necessary tokenized fields (`input_ids`, `attention_mask`) remain.

#### **Final Output**
- The `tokenized_datasets` object contains the processed dataset, ready for model training.

This setup ensures the dataset is efficiently formatted for fine-tuning GPT-2.


In [ ]:
model_checkpoint = "gpt2"  # o "gpt2-medium", "gpt2-large", ecc.

tokenizer = GPT2Tokenizer.from_pretrained(model_checkpoint)
# Imposta un token di padding se non definito
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': tokenizer.eos_token})

# Imposta una lunghezza massima (ad esempio 512, da valutare in base a GPT-2)
max_length = 256

def tokenize_function(examples):
    # Ritorna un unico dict con input_ids e attention_mask
    return tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=max_length
    )

tokenized_datasets = dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=dataset["train"].column_names
)

tokenized_datasets


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Map:   0%|          | 0/134 [00:00<?, ? examples/s]

Map:   0%|          | 0/34 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 134
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 34
    })
})

In [ ]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # Causal Language Modeling
)


In [ ]:
model = GPT2LMHeadModel.from_pretrained(model_checkpoint)

# Aggiungiamo eventuali token se abbiamo aggiunto un token di pad
model.resize_token_embeddings(len(tokenizer))


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Embedding(50257, 768)

### Evaluation Metrics and Text Postprocessing
This section defines the evaluation metrics and a helper function to preprocess text for evaluation.

#### **Evaluation Metrics**
- **ROUGE:** Measures overlap between n-grams, sequences, and longest common subsequences between predictions and references. Ideal for summarization and text generation tasks.
- **BLEU:** Evaluates the precision of generated words compared to reference texts, often used for translation tasks.
- **METEOR:** Focuses on synonym matching, stemming, and word order, providing a nuanced evaluation for generated text.

The metrics are loaded using the `evaluate` library, enabling easy integration into the training and evaluation workflow.

#### **Postprocessing Function**
- The `postprocess_text` function ensures predictions and references are properly formatted for accurate metric computation:
  - **Whitespace Cleanup:** Removes unnecessary spaces from both predictions and labels.
  - **Sentence Tokenization:** Segments predictions and references into sentences using NLTK's `sent_tokenize` to align with how ROUGE calculates scores.

This preprocessing ensures consistency and improves the reliability of the evaluation metrics.


In [ ]:
rouge_score = evaluate.load("rouge")
bleu_score  = evaluate.load("bleu")
meteor_score = evaluate.load("meteor")

def postprocess_text(preds, labels):
    """
    - Rimuove spazi superflui
    - Segmenta in frasi per calcolare ROUGE in modo corretto
    """
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]
    return preds, labels


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


### Compute Metrics Function for Model Evaluation
This function computes **ROUGE**, **BLEU**, and **METEOR** scores to evaluate model-generated text against reference labels.

#### **Processing Predictions and Labels**
- The function receives model `logits` (raw outputs) and `labels` (ground truth).
- **Handling Masked Tokens:**  
  - In **GPT-2**, `-100` is not typically used for ignored tokens (as in seq2seq models).
  - If present, it is replaced with the tokenizer’s pad token.
- **Argmax Decoding:**  
  - The highest probability token is selected for each position in the logits.
  - The resulting token IDs are converted into human-readable text using `batch_decode()`.

#### **Post-Processing**
- **Whitespace Cleanup & Sentence Splitting:**  
  - Predictions and labels are preprocessed using `postprocess_text()`.
  - This ensures proper tokenization for ROUGE and improves metric accuracy.

#### **Metric Computation**
- **ROUGE:**  
  - Measures n-gram overlap between predictions and references.
- **BLEU:**  
  - `evaluate` requires references in a **list of lists** format (`[[reference]]`).
  - Measures word sequence precision against references.
- **METEOR:**  
  - Considers synonym matching, stemming, and word order for better evaluation.

#### **Result Formatting**
- Scores are converted to percentages (`*100`) and rounded for readability.
- The final dictionary contains:
  - **ROUGE-1, ROUGE-2, ROUGE-L** (n-gram and longest common subsequence similarity)
  - **BLEU** (precision-based similarity)
  - **METEOR** (semantic matching)

This function ensures a **comprehensive evaluation** of model-generated text.


In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred

    # In GPT-2, di solito non usiamo -100 come mask (come in seq2seq),
    # ma se presente, lo sostituiamo con il pad_token_id
    labels[labels == -100] = tokenizer.pad_token_id

    # Argmax sui logits per ottenere la sequenza predetta
    predictions = np.argmax(logits, axis=-1)

    # Decodifica in stringhe
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Post-processing (rimozione spazi, split in frasi)
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    # Calcolo delle metriche
    # 1) ROUGE
    rouge_results = rouge_score.compute(
        predictions=decoded_preds,
        references=decoded_labels
    )
    # 2) BLEU
    # La metrica BLEU in `evaluate` richiede `references` come lista di liste
    bleu_results = bleu_score.compute(
        predictions=decoded_preds,
        references=[[lbl] for lbl in decoded_labels]
    )
    # 3) METEOR
    meteor_results = meteor_score.compute(
        predictions=decoded_preds,
        references=decoded_labels
    )

    # Organizza i risultati
    result = {}
    # ROUGE
    result["rouge1"] = round(rouge_results["rouge1"] * 100, 2)
    result["rouge2"] = round(rouge_results["rouge2"] * 100, 2)
    result["rougeL"] = round(rouge_results["rougeL"] * 100, 2)
    # BLEU
    result["bleu"] = round(bleu_results["bleu"] * 100, 2)
    # METEOR
    result["meteor"] = round(meteor_results["meteor"] * 100, 2)

    return result


### Training Configuration and Trainer Initialization
This section configures the training arguments and initializes the `Trainer` for fine-tuning GPT-2 on IFTTT-like tasks.

#### **Training Arguments Configuration**
- **Output Directory:**  
  - Saves model checkpoints in `./gpt2-ifttt`.
  - Overwrites existing checkpoints if `overwrite_output_dir=True`.
- **Evaluation & Checkpointing Strategy:**  
  - `evaluation_strategy="epoch"` → Evaluates the model at the end of each epoch.
  - `save_strategy="epoch"` → Saves model checkpoints at the end of each epoch.
- **Training Hyperparameters:**  
  - `num_train_epochs=30` → Trains for 30 epochs (adjust based on dataset size and performance).
  - `per_device_train_batch_size=4` → Processes 4 examples per batch (adjust based on GPU memory).
  - `per_device_eval_batch_size=4` → Uses the same batch size for evaluation.
- **Logging & Precision:**  
  - `logging_steps=50` → Logs progress every 50 steps.
  - `fp16=torch.cuda.is_available()` → Uses **mixed precision** (`fp16`) if a compatible GPU is available, improving efficiency.
  - `report_to="none"` → Disables logging to external tools like WandB.

#### **Trainer Initialization**
- The `Trainer` class simplifies training by handling:
  - **Model Training & Evaluation:** Uses the specified datasets.
  - **Data Collation:** Ensures correct batch formatting using `data_collator`.
  - **Tokenization:** Ensures consistency in input processing.
  - **Metric Computation:** If `compute_metrics` is provided, the model's performance is evaluated after each epoch.

This setup allows for an **automated and structured training process**, including evaluation and checkpointing.


In [ ]:
training_args = TrainingArguments(
    output_dir="./gpt2-ifttt",
    overwrite_output_dir=True,
    evaluation_strategy="epoch",   # Esegui evaluation alla fine di ogni epoca
    save_strategy="epoch",         # Salva un checkpoint a ogni epoca
    num_train_epochs=30,            # Cambia secondo le tue necessità
    per_device_train_batch_size=4, # Batch size, adattalo alla tua GPU
    per_device_eval_batch_size=4,
    logging_steps=50,
    fp16=torch.cuda.is_available(), # Usa half precision se possibile
    report_to="none",               # Disabilita WandB o altri logger
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,  # se vuoi, altrimenti None
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-32-8b440416c6ab>:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Bleu,Meteor
1,No log,1.917171,49.430000,23.730000,44.810000,36.320000,53.760000
2,1.496000,1.778151,52.810000,28.530000,48.260000,39.370000,57.540000
3,1.284000,1.782334,52.690000,29.530000,48.660000,40.720000,58.230000
4,1.284000,1.736189,55.320000,32.380000,51.010000,42.280000,61.380000
5,1.047300,1.804302,55.120000,33.430000,51.260000,43.310000,61.620000
6,0.854300,1.846444,54.640000,32.730000,51.230000,42.110000,61.240000
7,0.854300,1.919691,54.530000,31.610000,50.900000,42.400000,60.540000
8,0.684900,1.917195,55.240000,33.630000,51.690000,42.170000,61.420000
9,0.594400,1.957487,54.750000,33.930000,51.550000,41.050000,61.260000
10,0.594400,2.001669,55.450000,35.140000,52.630000,47.420000,62.400000


TrainOutput(global_step=1020, training_loss=0.47468644684436273, metrics={'train_runtime': 1498.1581, 'train_samples_per_second': 2.683, 'train_steps_per_second': 0.681, 'total_flos': 525196984320000.0, 'train_loss': 0.47468644684436273, 'epoch': 30.0})

In [ ]:
results = trainer.evaluate()
print("Final eval_loss:", results["eval_loss"])
print("Perplexity:", math.exp(results["eval_loss"]))


Final eval_loss: 2.3817808628082275
Perplexity: 10.824162057488907


In [ ]:
%cd ..


/content/ifttt-code-generator


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Define the path to your shared folder in Google Drive
shared_folder_path = "/content/drive/Shareddrives/NLPMODELS/"  # Replace with your actual shared drive and folder name

# Create the shared folder if it doesn't exist
!mkdir -p "{shared_folder_path}"

# Save the nl2sql_bart folder to your shared Google Drive folder
!cp -r gpt2-ifttt "{shared_folder_path}"

Mounted at /content/drive


In [ ]:
from transformers import pipeline

# Carica (o ri-carica) il modello fine-tunato
# Se hai salvato i checkpoint in `./gpt2-ifttt`, puoi caricare da lì
inference_model = GPT2LMHeadModel.from_pretrained("gpt2-ifttt/checkpoint-1020")
inference_tokenizer = GPT2Tokenizer.from_pretrained("gpt2-ifttt/checkpoint-1020")

# Pipeline di text generation
generator = pipeline(
    "text-generation",
    model=inference_model,
    tokenizer=inference_tokenizer,
    pad_token_id=inference_tokenizer.eos_token_id
)

# Esempio di prompt: solo la "descrizione"
prompt = "Description:\nCreate an applet that sends a message between 9pm and 12pm.\n###\nCode:\n"

results = generator(prompt, max_length=256, num_return_sequences=1)
print(results[0]["generated_text"])


Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Description:
Create an applet that sends a message between 9pm and 12pm.
###
Code:
var hour = Meta.currentUserTime.hour()    if (hour > 18 && hour < 9) {   Gmail.sendAnEmail.skip("Too late") }   if (hour > 19 && hour < 12) {   Gmail.sendAnEmail.skip("Too early") }   else {   Gmail.sendAnEmail.skip("I already know") }    else {   Gmail.sendAnEmail.skip("I already know") } }     return; }    if (hour < 9 || hour > 19) {   Gmail.sendAnEmail.skip("I already know") }   else {   Gmail.sendAnEmail.skip("I already know") }   return; }    if (hour > 18 || hour > 12) {   Gmail:sendAnEmail.skip("Too late") }   return; }   if (hour > 19) {   Gmail.sendAnEmail.skip("Too early") }   return; }   return; }
